# Libs

In [1]:
import xarray as xr
import lexcube
import segyio
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pathlib
from segysak.segy import segy_loader, well_known_byte_locs, segy_writer, segy_header_scrape
from tqdm import tqdm

warnings.filterwarnings('ignore')

# Data uploading

In [2]:
df8 = pd.read_csv(r"C:\jupyter\SPP\inputoutput\general_logs\logs8_ntd_v5.csv")
df8_xy = df8.groupby('well')[['xmean', 'ymean']].first().reset_index()
# df8_xy

In [3]:
# file_path = pathlib.Path(r"C:\Petrel\gr_cube\GR_cube_Bal8.sgy")
# b8_crop = segyio.open(file_path)
# df_cube = segy_header_scrape(file_path, chunk=10000)
# df_cube.to_parquet(r"C:\Petrel\gr_cube\GR_cube_Bal8.gzip")

In [ ]:
b8_crop = segyio.open(r"C:\Petrel\gr_cube\GR_cube_Bal8.sgy")
b8_crop.ilines, b8_crop.xlines, b8_crop.samples, len(b8_crop.ilines), len(b8_crop.xlines), len(b8_crop.samples)

In [ ]:
v = segyio.open(r"C:\Petrel\gr_cube\Vsh_clavier_HDOBN_T_exact_value_renorm.sgy")
v.ilines, v.xlines, v.samples, len(v.ilines), len(v.xlines), len(v.samples)

In [6]:
df_cube_crop = pd.read_parquet(r"C:\Petrel\gr_cube\GR_cube_Bal8.gzip")
df_xy_lines_crop = df_cube_crop[['CDP_X', 'CDP_Y', 'INLINE_3D', 'CROSSLINE_3D']]
df_xy_lines_crop.columns = df_xy_lines_crop.columns.str.lower()


In [7]:
df_cube = pd.read_parquet(r"C:\Petrel\gr_cube\GR_cube_HDOBN_Z.gzip")
df_xy_lines = df_cube[['CDP_X', 'CDP_Y', 'INLINE_3D', 'CROSSLINE_3D']]
df_xy_lines.columns = df_xy_lines.columns.str.lower()

# Well selection

## crop

In [ ]:
b02 = df8[df8['well'] == 'B02']
df_xy_lines_v2 = df_xy_lines_crop[((df_xy_lines_crop.inline_3d >=3900) & (df_xy_lines_crop.inline_3d <= 6900)) &
                                   ((df_xy_lines_crop.inline_3d >= 3900) & (df_xy_lines_crop.inline_3d <= 6900)) ]
df_xy_lines_v2['cdp_x'] = df_xy_lines_v2['cdp_x']/100
df_xy_lines_v2['cdp_y'] = df_xy_lines_v2['cdp_y']/100
margin = 100
well_xmax = b02['xmean']+margin
well_xmin = b02['xmean']-margin
well_ymax = b02['ymean']+margin
well_ymin = b02['ymean']-margin
plt.scatter(b02['xmean'], b02['ymean'], c='r')
plt.scatter(well_xmax, well_ymax, c='g')
plt.scatter(well_xmin, well_ymin, c='b')
plt.scatter(well_xmax, well_ymin, c='y')
plt.scatter(well_xmin, well_ymax, c='orange')
df_xy_lines_v3 = df_xy_lines_v2[((df_xy_lines_v2['cdp_x'] > well_xmin.values[0]) & (df_xy_lines_v2['cdp_x'] < well_xmax.values[0])) & 
                                ((df_xy_lines_v2['cdp_y'] > well_ymin.values[0]) & (df_xy_lines_v2['cdp_y'] < well_ymax.values[0]))]
in_max, xn_max, in_min, xn_min = df_xy_lines_v3.inline_3d.max(), df_xy_lines_v3.crossline_3d.max(), df_xy_lines_v3.inline_3d.min(), df_xy_lines_v3.crossline_3d.min()
plt.scatter(df_xy_lines_v3['cdp_x'], df_xy_lines_v3['cdp_y'], c='gray', s=1)

In [ ]:
df_inline_lst = []
for i in df_xy_lines_crop.inline_3d.unique()[::50]:
    data = df_xy_lines_crop[df_xy_lines_crop.inline_3d == i]
    df_inline_lst.append(data)
df_inline = pd.concat(df_inline_lst)
df_inline = df_inline[(df_inline.inline_3d >=3900) & (df_inline.inline_3d <= 6900)]

df_xline_lst = []
for i in df_xy_lines_crop.crossline_3d.unique()[::50]:
    data = df_xy_lines_crop[df_xy_lines_crop.crossline_3d == i]
    df_xline_lst.append(data)
df_xline = pd.concat(df_xline_lst)
df_xline = df_xline[(df_xline.inline_3d >= 3900) & (df_xline.inline_3d <= 6900)]

def numbers_of_inlines(df_inline):
    df_lst = []
    for i in df_inline.inline_3d.unique():
        text_pos = df_inline[df_inline.inline_3d == i].iloc[-1].values[:2]
        text = df_inline[df_inline.inline_3d == i].iloc[-1].values[2]
        df = pd.DataFrame({'x_txt':text_pos[0]/100, 'y_txt':text_pos[1]/100, 'inline_n':int(text)}, index=[0])
        df_lst.append(df)
    df_inline_txt = pd.concat(df_lst).reset_index(drop=True)
    return df_inline_txt
def numbers_of_xlines(df_xline):
    df_lst = []
    for i in df_xline.crossline_3d.unique():
        text_pos = df_xline[df_xline.crossline_3d == i].iloc[-1].values[:2]
        text = df_xline[df_xline.crossline_3d == i].iloc[-1].values[3]
        df = pd.DataFrame({'x_txt':text_pos[0]/100, 'y_txt':text_pos[1]/100, 'xline_n':int(text)}, index=[0])
        df_lst.append(df)
    df_xline_txt = pd.concat(df_lst).reset_index(drop=True)
    return df_xline_txt

df_inline_txt = numbers_of_inlines(df_inline)
df_xline_txt = numbers_of_xlines(df_xline)

plt.figure(figsize=(15, 7))
plt.scatter(df_inline.cdp_x/100, df_inline.cdp_y/100, s=1)
plt.scatter(df_xline.cdp_x/100, df_xline.cdp_y/100, s=1)
plt.scatter(df8_xy.xmean, df8_xy.ymean, s=15, c='g')
plt.scatter(b02.xmean, b02.ymean, s=30, c='purple')
plt.legend(['inline', 'crossline', 'well'])
for i, row in df_inline_txt.iterrows():
    plt.text(row.x_txt, row.y_txt, int(row.inline_n), fontsize=6, ha='left')
for i, row in df_xline_txt.iterrows():
    plt.text(row.x_txt, row.y_txt, int(row.xline_n), fontsize=8, ha='left')

In [ ]:
b02.columns
tvd_max = round(b02['tvd_scs'].max(),0)
tvd_min = round(b02['tvd_scs'].min(),0)
tvd_max, tvd_min

tvd_max_idx = np.where(b8_crop.samples.round(0) == tvd_max)[0][0]
tvd_min_idx = np.where(b8_crop.samples.round(0) == tvd_min)[0][0]
print(in_max, xn_max, in_min, xn_min, tvd_min_idx,  tvd_max_idx)

In [ ]:
b8_crop.ilines, b8_crop.xlines, b8_crop.samples, len(b8_crop.ilines), len(b8_crop.xlines), len(b8_crop.samples)

In [ ]:
cc = 0
for idx, row in df_xy_lines_crop[df_xy_lines_crop.inline_3d == in_max].iterrows():
    cc +=1
    if row['crossline_3d'] == 2188:
        print(idx,cc)
        break

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))
test = b8_crop.iline[5096][789][tvd_min_idx-60:tvd_max_idx-150]
indices = np.arange(test.shape[0])
ax[0].plot(test, indices)
ax[0].invert_yaxis()

test2 = df8[df8['well'] == 'B02']
ax[1].plot(test2['gr_n'], test2['tvd_scs'])
ax[1].invert_yaxis()

## base

In [ ]:
b02 = df8[df8['well'] == 'B02']
df_xy_lines_v2 = df_xy_lines[((df_xy_lines.inline_3d >=3900) & (df_xy_lines.inline_3d <= 6900)) & ((df_xy_lines.inline_3d >= 3900) & (df_xy_lines.inline_3d <= 6900)) ]
df_xy_lines_v2['cdp_x'] = df_xy_lines_v2['cdp_x']/100
df_xy_lines_v2['cdp_y'] = df_xy_lines_v2['cdp_y']/100
margin = 100
well_xmax = b02['xmean']+margin
well_xmin = b02['xmean']-margin
well_ymax = b02['ymean']+margin
well_ymin = b02['ymean']-margin
plt.scatter(b02['xmean'], b02['ymean'], c='r')
plt.scatter(well_xmax, well_ymax, c='g')
plt.scatter(well_xmin, well_ymin, c='b')
plt.scatter(well_xmax, well_ymin, c='y')
plt.scatter(well_xmin, well_ymax, c='orange')
df_xy_lines_v3 = df_xy_lines_v2[((df_xy_lines_v2['cdp_x'] > well_xmin.values[0]) & (df_xy_lines_v2['cdp_x'] < well_xmax.values[0])) & 
                                ((df_xy_lines_v2['cdp_y'] > well_ymin.values[0]) & (df_xy_lines_v2['cdp_y'] < well_ymax.values[0]))]
in_max, xn_max, in_min, xn_min = df_xy_lines_v3.inline_3d.max(), df_xy_lines_v3.crossline_3d.max(), df_xy_lines_v3.inline_3d.min(), df_xy_lines_v3.crossline_3d.min()
plt.scatter(df_xy_lines_v3['cdp_x'], df_xy_lines_v3['cdp_y'], c='gray', s=1)

In [ ]:
b02.columns
tvd_max = round(b02['tvd_scs'].max(),0)
tvd_min = round(b02['tvd_scs'].min(),0)
tvd_max, tvd_min

tvd_max_idx = np.where(v.samples == tvd_max)[0][0]
tvd_min_idx = np.where(v.samples == tvd_min)[0][0]
print(tvd_max_idx, tvd_min_idx)

In [ ]:
df_xy_lines_v2 = df_xy_lines[((df_xy_lines.inline_3d >=3900) & (df_xy_lines.inline_3d <= 6900)) & ((df_xy_lines.inline_3d >= 3900) & (df_xy_lines.inline_3d <= 6900)) ]
df_xy_lines_v2['cdp_x'] = df_xy_lines_v2['cdp_x']/100
df_xy_lines_v2['cdp_y'] = df_xy_lines_v2['cdp_y']/100
plt.scatter(df_xy_lines_v2.cdp_x, df_xy_lines_v2.cdp_y, s=1)
plt.scatter(b02.xmean, b02.ymean, s=10)

In [ ]:
df_inline_lst = []
for i in df_xy_lines.inline_3d.unique()[::50]:
    data = df_xy_lines[df_xy_lines.inline_3d == i]
    df_inline_lst.append(data)
df_inline = pd.concat(df_inline_lst)
df_inline = df_inline[(df_inline.inline_3d >=3900) & (df_inline.inline_3d <= 6900)]

df_xline_lst = []
for i in df_xy_lines.crossline_3d.unique()[::50]:
    data = df_xy_lines[df_xy_lines.crossline_3d == i]
    df_xline_lst.append(data)
df_xline = pd.concat(df_xline_lst)
df_xline = df_xline[(df_xline.inline_3d >= 3900) & (df_xline.inline_3d <= 6900)]

def numbers_of_inlines(df_inline):
    df_lst = []
    for i in df_inline.inline_3d.unique():
        text_pos = df_inline[df_inline.inline_3d == i].iloc[-1].values[:2]
        text = df_inline[df_inline.inline_3d == i].iloc[-1].values[2]
        df = pd.DataFrame({'x_txt':text_pos[0]/100, 'y_txt':text_pos[1]/100, 'inline_n':int(text)}, index=[0])
        df_lst.append(df)
    df_inline_txt = pd.concat(df_lst).reset_index(drop=True)
    return df_inline_txt
def numbers_of_xlines(df_xline):
    df_lst = []
    for i in df_xline.crossline_3d.unique():
        text_pos = df_xline[df_xline.crossline_3d == i].iloc[-1].values[:2]
        text = df_xline[df_xline.crossline_3d == i].iloc[-1].values[3]
        df = pd.DataFrame({'x_txt':text_pos[0]/100, 'y_txt':text_pos[1]/100, 'xline_n':int(text)}, index=[0])
        df_lst.append(df)
    df_xline_txt = pd.concat(df_lst).reset_index(drop=True)
    return df_xline_txt

df_inline_txt = numbers_of_inlines(df_inline)
df_xline_txt = numbers_of_xlines(df_xline)

plt.figure(figsize=(15, 7))
plt.scatter(df_inline.cdp_x/100, df_inline.cdp_y/100, s=1)
plt.scatter(df_xline.cdp_x/100, df_xline.cdp_y/100, s=1)
plt.scatter(df8_xy.xmean, df8_xy.ymean, s=15, c='r')
plt.scatter(b02.xmean, b02.ymean, s=50, c='purple')
plt.legend(['inline', 'crossline', 'well'])
for i, row in df_inline_txt.iterrows():
    plt.text(row.x_txt, row.y_txt, int(row.inline_n), fontsize=6, ha='left')
for i, row in df_xline_txt.iterrows():
    plt.text(row.x_txt, row.y_txt, int(row.xline_n), fontsize=8, ha='left')

In [ ]:
in_max, in_min, xn_max, xn_min, tvd_min_idx,tvd_max_idx

# 3d cube

In [ ]:
# B02
# in_max, xn_max, in_min, xn_min
# tvd_max_idx, tvd_min_idx

iline_min = in_min
iline_max = in_max
xline_min = xn_min 
xline_max = xn_max
sample_min = tvd_min_idx-50
sample_max = tvd_max_idx+50

offset_val = 0

ils = np.arange(iline_min - offset_val, iline_max + offset_val)
# xls = np.arange(xline_min - offset_val, xline_max + offset_val)
xls = np.arange(xline_min - offset_val, xline_max + offset_val) - v.xlines.min()
samples_dim = v.samples[sample_min:sample_max] # ignore out of bal8 values (manual adjust)

three_d_array = np.empty((len(ils), len(xls), len(samples_dim)))
# three_d_array = np.empty((ils, len(xls), len(samples_dim)))

for il_idx, il in enumerate(ils):
    for xl_idx, xl in enumerate(xls):
        gr = v.iline[il][xl][sample_min:sample_max] # ignore out of bal8 values (manual adjust)
#         gr_adj = (gr - 43.948) / (69.331 - 43.948) * (70.384 - 36.241) + 36.241
#         gr_adj = np.where(gr_adj < 0, 0, gr_adj)
#         igr = (gr_adj - 33) / (86-33)
#         vsh = 1.7 - (3.38-(igr+0.7)**2)**0.5
        gr = np.where(gr == -1, -9999, gr)
        gr = np.where(gr > 0.2, 1, gr)
#         vsh = np.where(vsh < 0, 0, vsh)
        three_d_array[il_idx, xl_idx, :] = gr

data_array = xr.DataArray(three_d_array, coords={
    'iline': ils,
    'xline': np.arange(xline_min - offset_val, xline_max + offset_val),
    'z':samples_dim,
})

data_array = data_array.transpose('xline','z','iline')

w = lexcube.Cube3DWidget(data_array,cmap="viridis_r", vmin=0, vmax=1)
w

In [20]:
# w.show_sliders()

# 3d cube crop

In [21]:
# np.set_printoptions(suppress=True)
# data_array

In [22]:
# b8_crop
# iline_min = in_min
# iline_max = in_max
# xline_min = xn_min 
# xline_max = xn_max
# sample_min = tvd_min_idx-50
# sample_max = tvd_max_idx

# offset_val = 0

# ils = np.arange(iline_min - offset_val, iline_max + offset_val)
# # xls = np.arange(xline_min - offset_val, xline_max + offset_val)
# xls = np.arange(xline_min - offset_val, xline_max + offset_val) - b8_crop.xlines.min()
# samples_dim = b8_crop.samples[sample_min:sample_max] # ignore out of bal8 values (manual adjust)

# three_d_array = np.empty((len(ils), len(xls), len(samples_dim)))
# # three_d_array = np.empty((ils, len(xls), len(samples_dim)))

# for il_idx, il in enumerate(ils):
#     for xl_idx, xl in enumerate(xls):
#         gr = b8_crop.iline[il][xl][sample_min:sample_max] # ignore out of bal8 values (manual adjust)
#         gr = np.where(gr == 0, -9999, gr)
#         # print(il, xl, gr)
#         gr_adj = np.where(gr == -9999, gr, ((gr - 43.948) / (69.331 - 43.948) * (70.384 - 36.241) + 36.241))
#         # gr_adj = np.where(gr_adj < 0, 0, gr_adj)
#         igr = np.where(gr == -9999, gr, ((gr_adj - 33) / (86-33)))
#         vsh = np.where(gr == -9999, gr, (1.7 - (3.38-(igr+0.7)**2)**0.5))
#         # vsh = np.where(gr == -1, -9999, vsh)
#         # gr = np.where(gr <=0.2, gr, 1)
#         vsh = np.where(vsh <= 0.5, vsh, 1)
#         cond1 = vsh != -9999
#         cond2 = vsh < 0
#         combine = cond1 & cond2
#         vsh = np.where(combine, 0, vsh)
#         three_d_array[il_idx, xl_idx, :] = vsh
#         # three_d_array[il_idx, xl_idx, :] = gr

# data_array = xr.DataArray(three_d_array, coords={
#     'iline': ils,
#     'xline': np.arange(xline_min - offset_val, xline_max + offset_val),
#     'z':samples_dim,
# })

# data_array = data_array.transpose('xline','z','iline')

# w = lexcube.Cube3DWidget(data_array,cmap="viridis_r", vmin=0, vmax=1)
# w

# Final function

In [ ]:
df8[(df8.phit_flag == 1) & (df8.well.str.startswith('D'))].well.unique()

In [24]:
# отредактировать нейминг внутри функции!!!
def inline_xline_well_coord(well_df, df_xy_lines_crop, wellname):
    b02 = well_df[well_df['well'] == wellname]
    df_xy_lines_v2 = df_xy_lines_crop[((df_xy_lines_crop.inline_3d >=3900) & (df_xy_lines_crop.inline_3d <= 6900)) &
                                    ((df_xy_lines_crop.inline_3d >= 3900) & (df_xy_lines_crop.inline_3d <= 6900)) ]
    df_xy_lines_v2['cdp_x'] = df_xy_lines_v2['cdp_x']/100
    df_xy_lines_v2['cdp_y'] = df_xy_lines_v2['cdp_y']/100
    margin = 150
    well_xmax = b02['xmean']+margin
    well_xmin = b02['xmean']-margin
    well_ymax = b02['ymean']+margin
    well_ymin = b02['ymean']-margin

    df_xy_lines_v3 = df_xy_lines_v2[((df_xy_lines_v2['cdp_x'] > well_xmin.values[0]) & (df_xy_lines_v2['cdp_x'] < well_xmax.values[0])) & 
                                    ((df_xy_lines_v2['cdp_y'] > well_ymin.values[0]) & (df_xy_lines_v2['cdp_y'] < well_ymax.values[0]))]
    in_max, xn_max, in_min, xn_min = df_xy_lines_v3.inline_3d.max(), df_xy_lines_v3.crossline_3d.max(), df_xy_lines_v3.inline_3d.min(), df_xy_lines_v3.crossline_3d.min()

    tvd_max = round(b02['tvd_scs'].max(),0)
    tvd_min = round(b02['tvd_scs'].min(),0)

    tvd_max_idx = np.where(b8_crop.samples.round(0) == tvd_max)[0][0]
    tvd_min_idx = np.where(b8_crop.samples.round(0) == tvd_min)[0][0]
    result = {
        'in_max': in_max,
        'in_min': in_min,
        'xn_max': xn_max,
        'xn_min': xn_min,
        'tvd_min_idx': tvd_min_idx,
        'tvd_max_idx': tvd_max_idx
    }
    return result
# result = inline_xline_well_coord(df8, df_xy_lines_crop, 'B02')
def cube_data(dict_result, gr_cube, cutoff):
    iline_min = dict_result['in_min']
    iline_max = dict_result['in_max']
    xline_min = dict_result['xn_min']
    xline_max = dict_result['xn_max']
    sample_min = dict_result['tvd_min_idx']-100
    sample_max = dict_result['tvd_max_idx']+100

    offset_val = 0

    ils = np.arange(iline_min - offset_val, iline_max + offset_val)
    xls = np.arange(xline_min - offset_val, xline_max + offset_val) - b8_crop.xlines.min()
    samples_dim = b8_crop.samples[sample_min:sample_max] # ignore out of bal8 values (manual adjust)

    three_d_array = np.empty((len(ils), len(xls), len(samples_dim)))

    for il_idx, il in enumerate(ils):
        for xl_idx, xl in enumerate(xls):
            gr = gr_cube.iline[il][xl][sample_min:sample_max] # ignore out of bal8 values (manual adjust)
            gr = np.where(gr == 0, -9999, gr)
            gr_adj = np.where(gr == -9999, gr, ((gr - 43.948) / (69.331 - 43.948) * (70.384 - 36.241) + 36.241))
            igr = np.where(gr == -9999, gr, ((gr_adj - 33) / (86-33)))
            vsh = np.where(gr == -9999, gr, (1.7 - (3.38-(igr+0.7)**2)**0.5))
            vsh = np.where(vsh <= cutoff, vsh, 1)
            cond1 = vsh != -9999
            cond2 = vsh < 0
            combine = cond1 & cond2
            vsh = np.where(combine, 0, vsh)
            three_d_array[il_idx, xl_idx, :] = vsh

    data_array = xr.DataArray(three_d_array, coords={
        'iline': ils,
        'xline': np.arange(xline_min - offset_val, xline_max + offset_val),
        'z':samples_dim,
    })

    data_array = data_array.transpose('xline','z','iline')
    return data_array

In [ ]:
df_lst = []
for wellname in tqdm([  'C15', 'C16', 'C17', 'C18', 'B08',
                        'B08Z', 'B10', 'B11', 'B12', 'B13ST2', 'B14Z', 'B16Y',
                        'D23X', 'D24', 'D25'
                        ]):
    result = inline_xline_well_coord(df8, df_xy_lines_crop, wellname)
    data_array = cube_data(result, b8_crop, 0.5)
    condition1 = data_array > 0
    condition2 = data_array <= 0.2
    combined_condition = condition1 & condition2
    count = combined_condition.sum().item()
    df = pd.DataFrame({'well':wellname, 'count':count}, index=[0])
    df_lst.append(df)
df_count = pd.concat(df_lst).reset_index(drop=True)

In [ ]:
khtst = df8.groupby('well')['khtst'].first().reset_index()
test = df_count.set_index('well').join(khtst.set_index('well'))
sns.scatterplot(data=test, x='khtst', y='count')
plt.xlim(0, 35000)
plt.ylim(5000, 20000)
# plt.yscale('log')
# plt.xscale('log')
for idx, txt in enumerate(test.index):
    plt.text(test['khtst'][idx], test['count'][idx], txt, fontsize=8)

In [ ]:
# result_well = inline_xline_well_coord(df8, df_xy_lines_crop, 'C16')
# data_array_well = cube_data(result_well, b8_crop, 0.2)
# w = lexcube.Cube3DWidget(data_array_well,cmap="viridis_r", vmin=0, vmax=1)
# w

In [ ]:
# result_well = inline_xline_well_coord(df8, df_xy_lines_crop, 'B13ST2')
# data_array_well = cube_data(result_well, b8_crop, 0.2)
# w = lexcube.Cube3DWidget(data_array_well,cmap="viridis_r", vmin=0, vmax=1)
# w

In [ ]:
# result_well = inline_xline_well_coord(df8, df_xy_lines_crop, 'D25')
# data_array_well = cube_data(result_well, b8_crop, 0.2)
# w = lexcube.Cube3DWidget(data_array_well,cmap="viridis_r", vmin=0, vmax=1)
# w